In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import json
import os
import sys


In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def check_exists_by_xpath(driver, xpath):
    
    driver.implicitly_wait(1)
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        driver.implicitly_wait(30)
        return False
    driver.implicitly_wait(30)
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
with open(r'./class_schedules/25 January - 31 January 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [7]:
temp = []

c = search_courses_by_date('28', course_schedule)
c = search_courses_by_name('Aerial', c)
c = search_courses_by_teacher('Marcus', c)
c = search_courses_by_time('PM', c)
temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 28 Jan Thu
-----------------------

Name: Aerial Yoga 空中瑜珈
Teacher: Marcus
Time: 08:45 - 09:45 PM




In [8]:
master_counter = 0

In [9]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [10]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter

    stride =60 / 30 * 10 * len(courses_to_be_booked)


    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    master_counter = master_counter + 1

    if click_book_this_class_now(driver, wait):
        log_str = "{},{} by {},BOOKED,{}\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(r'./log_try_booking_a_class_that_was_full.csv', "a") as write_file:
            write_file.write(log_str)
    else:
        
        log_str = "{},{} by {},FULL,{}\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
#         print("master count = {}, stride = {}".format(master_counter, stride))
        if master_counter % stride == 0 and master_counter > 0:
            
            with open(r'./log_try_booking_a_class_that_was_full.csv', "a") as write_file:
                write_file.write(log_str)
        
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    


In [11]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(r'./log_try_booking_a_class_that_was_full.csv', "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=20)
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


 
1,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:03:44.389967

2,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:04:35.025677

3,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:05:25.687086

4,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:06:16.324874

5,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:07:06.956685

6,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:07:57.573426

7,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:08:48.227017

8,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:09:38.841971

9,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:10:29.468565

10,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:11:20.092517

11,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:12:10.753656

12,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:13:01.387072

13,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:13:52.033452

14,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:14:42.652469

15,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:15:33.261669

16,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 22:16:23.887276


131,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:53:45.782634

132,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:54:36.876791

133,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:55:27.625772

134,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:56:18.276097

135,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:57:08.929318

136,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:57:59.590532

137,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:58:50.223529

138,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-27 23:59:40.873188

139,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:00:31.523684

140,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:01:22.181860

141,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:02:12.830337

142,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:03:03.466503

143,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:03:54.103613

144,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:04:44.724506

145,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 00:05:35.350163

146,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021

260,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:42:41.681358

261,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:43:32.351127

262,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:44:22.983332

263,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:45:13.598519

264,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:46:04.211556

265,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:46:54.821027

266,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:47:45.436802

267,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:48:36.074056

268,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:49:26.712920

269,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:50:17.325970

270,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:51:07.967005

271,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:51:59.897250

272,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:52:51.756790

273,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:53:43.814857

274,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 01:54:35.889389

275,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021

389,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:32:35.331859

390,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:33:26.017612

391,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:34:16.658881

392,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:35:07.323542

393,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:35:58.046386

394,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:37:51.000800

395,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:38:41.840408

396,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:39:32.537653

397,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:40:23.218097

398,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:41:13.977743

399,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:42:04.696559

400,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:42:55.393984

401,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:43:46.065692

402,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:44:36.804453

403,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 03:45:27.487061

404,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021

...

1160,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:24:41.311939

1161,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:25:32.728107

1162,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:26:23.651535

1163,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:27:14.333976

1164,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:28:05.109809

1165,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:28:56.187984

1166,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:29:47.608150

1167,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:30:38.210172

1168,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:31:29.014682

1169,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:32:19.842827

1170,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:33:11.080207

1171,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:34:02.268162

1172,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:34:53.142827

1173,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:35:44.229233

1174,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 14:36:35.151406

1175,Aerial Yoga 空中瑜珈 by 

1287,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:12:14.856758

1288,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:13:06.014419

1289,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:13:57.276728

1290,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:14:48.714260

1291,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:15:39.285930

1292,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:16:29.988594

1293,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:17:20.568822

1294,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:18:11.204574

1295,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:19:01.867667

1296,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:19:52.472655

1297,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:20:43.086662

1298,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:21:33.785579

1299,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:22:24.490557

1300,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:23:15.493617

1301,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 16:24:06.171220

1302,Aerial Yoga 空中瑜珈 by 

1414,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 17:59:27.848011

1415,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:00:18.459311

1416,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:01:09.152203

1417,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:01:59.738299

1418,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:02:50.318212

1419,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:03:40.884400

1420,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:04:31.489175

1421,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:05:22.103820

1422,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:06:12.695598

1423,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:07:03.279971

1424,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:07:53.860277

1425,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:08:44.443304

1426,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:09:35.039085

1427,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:10:25.643680

1428,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 18:11:16.231334

1429,Aerial Yoga 空中瑜珈 by 

1541,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:46:34.036860

1542,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:47:24.633522

1543,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:48:15.209709

1544,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:49:05.802281

1545,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:49:56.360103

1546,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:50:47.052183

1547,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:51:37.640981

1548,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:52:28.202151

1549,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:53:18.785870

1550,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:54:09.397552

1551,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:54:59.957187

1552,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:55:50.558268

1553,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:56:41.160772

1554,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:57:31.725264

1555,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 19:58:22.388059

1556,Aerial Yoga 空中瑜珈 by 

1668,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:33:39.879115

1669,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:34:30.485918

1670,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:35:21.113678

1671,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:36:11.718567

1672,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:37:02.303887

1673,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:37:52.893128

1674,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:38:43.484624

1675,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:39:34.058756

1676,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:40:24.649552

1677,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:41:15.329929

1678,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:42:05.901417

1679,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:42:56.473034

1680,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:43:47.050862

1681,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:44:37.622670

1682,Aerial Yoga 空中瑜珈 by Marcus,FULL,2021-01-28 21:45:28.237861

1683,Aerial Yoga 空中瑜珈 by 